task name: Resolve LatentDirichletAllocation task

task description: Implement the provided C# code as a dotnet interactive notebook

In [ ]:
#r "nuget: Microsoft.ML, 1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;

In [ ]:
var mlContext = new MLContext();

In [ ]:
private class TextData
{
    public string Text { get; set; }
}

var samples = new List<TextData>(){
    new TextData(){ Text = "ML.NET's LatentDirichletAllocation API computes topic models." },
    new TextData(){ Text = "ML.NET's LatentDirichletAllocation API is the best for topic models." },
    new TextData(){ Text = "I like to eat broccoli and bananas." },
    new TextData(){ Text = "I eat bananas for breakfast." },
    new TextData(){ Text = "This car is expensive compared to last week's price." },
    new TextData(){ Text = "This car was $X last week." },
};

var dataview = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var pipeline = mlContext.Transforms.Text.NormalizeText("NormalizedText", "Text")
    .Append(mlContext.Transforms.Text.TokenizeIntoWords("Tokens", "NormalizedText"))
    .Append(mlContext.Transforms.Text.RemoveDefaultStopWords("Tokens"))
    .Append(mlContext.Transforms.Conversion.MapValueToKey("Tokens"))
    .Append(mlContext.Transforms.Text.ProduceNgrams("Tokens"))
    .Append(mlContext.Transforms.Text.LatentDirichletAllocation("Features", "Tokens", numberOfTopics: 3));

var transformer = pipeline.Fit(dataview);

In [ ]:
private class TransformedTextData : TextData
{
    public float[] Features { get; set; }
}

private static void PrintLdaFeatures(TransformedTextData prediction)
{
    for (int i = 0; i < prediction.Features.Length; i++)
    {
        Console.Write($"{prediction.Features[i]:F4}  ");
    }
    Console.WriteLine();
}

var predictionEngine = mlContext.Model.CreatePredictionEngine<TextData, TransformedTextData>(transformer);
PrintLdaFeatures(predictionEngine.Predict(samples[0]));
PrintLdaFeatures(predictionEngine.Predict(samples[1]));